# Tensorflow Object Detection Inference


In [1]:
!pip install -U --pre tensorflow=="2.2.0"

     |████████████████████████████████| 516.2MB 31kB/s 
     |████████████████████████████████| 3.0MB 46.5MB/s 
     |████████████████████████████████| 460kB 44.1MB/s 
  Found existing installation: tensorboard 2.4.1
    Uninstalling tensorboard-2.4.1:
      Successfully uninstalled tensorboard-2.4.1
  Found existing installation: tensorflow-estimator 2.4.0
    Uninstalling tensorflow-estimator-2.4.0:
      Successfully uninstalled tensorflow-estimator-2.4.0
  Found existing installation: tensorflow 2.4.1
    Uninstalling tensorflow-2.4.1:
      Successfully uninstalled tensorflow-2.4.1


## Install Object Detection API

In [2]:
!git clone --depth 1 https://github.com/tensorflow/models

Cloning into 'models'...
remote: Enumerating objects: 2449, done.
remote: Counting objects: 100% (2449/2449), done.
remote: Compressing objects: 100% (2042/2042), done.
remote: Total 2449 (delta 593), reused 1334 (delta 380), pack-reused 0
Receiving objects: 100% (2449/2449), 30.84 MiB | 31.43 MiB/s, done.
Resolving deltas: 100% (593/593), done.


In [3]:
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install .

Processing /content/models/research
  Created wheel for object-detection: filename=object_detection-0.1-cp37-none-any.whl size=1624302 sha256=8d4a0e548413b9a914c0773afa902e6e181d5e8914ec3c1f5733999bff9a701b
  Stored in directory: /tmp/pip-ephem-wheel-cache-qsaac6a5/wheels/94/49/4b/39b051683087a22ef7e80ec52152a27249d1a644ccf4e442ea
  Created wheel for avro-python3: filename=avro_python3-1.10.1-cp37-none-any.whl size=43735 sha256=267afea0914ea11fa0baf9636e240946708da73d2a0b19c9f99290781b79a643
  Stored in directory: /root/.cache/pip/wheels/65/fe/90/20d6d6d97223d80d20cb390be636619c536edab5658c12bdba
  Created wheel for future: filename=future-0.18.2-cp37-none-any.whl size=491058 sha256=28e36ce9d091035d84fd070259859a74782fe4da467826605216cc54ec966177
  Stored in directory: /root/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
  Created wheel for dill: filename=dill-0.3.1.1-cp37-none-any.whl size=78532 sha256=c60e6713a1f2ad5c188daa2bb8db1e1fd4c771e57945d40ea318

ERROR: multiprocess 0.70.11.1 has requirement dill>=0.3.3, but you'll have dill 0.3.1.1 which is incompatible.
ERROR: google-colab 1.0.0 has requirement requests~=2.23.0, but you'll have requests 2.25.1 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: apache-beam 2.28.0 has requirement avro-python3!=1.9.2,<1.10.0,>=1.8.1, but you'll have avro-python3 1.10.1 which is incompatible.


## Imports and setup

In [4]:
import os
import io
import matplotlib
import matplotlib.pyplot as plt
import scipy.misc
import numpy as np
from six import BytesIO
from PIL import Image, ImageDraw, ImageFont
from sklearn.metrics.pairwise import euclidean_distances, cosine_similarity
import time

import tensorflow as tf

from object_detection.utils import label_map_util, config_util
from object_detection.utils import visualization_utils as viz_utils

%matplotlib inline

from lxml import etree
import xml.etree.ElementTree as ET

## Connect with drive

In [5]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


## Load model

In [6]:
ROOT = 'gdrive/MyDrive/MIA2/landmark_detections'
PATH_TO_SAVED_MODEL = os.path.join(ROOT, 'models', 'saved_model')
PATH_TO_LABELS = os.path.join(ROOT, 'landmarks_label_map.pbtxt')

print('Model path {}'.format(PATH_TO_SAVED_MODEL))
print('Label path {}'.format(PATH_TO_LABELS))

Model path gdrive/MyDrive/MIA2/landmark_detections/models/saved_model
Label path gdrive/MyDrive/MIA2/landmark_detections/landmarks_label_map.pbtxt


In [7]:
# Load saved model and build the detection function
detect_fn = tf.saved_model.load(PATH_TO_SAVED_MODEL)
print('Model loaded')

Model loaded


## Load label map

In [8]:
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)
category_index

{1: {'id': 1, 'name': 'mark'}}

## Run detections

In [9]:
def load_image_into_numpy_array(path):
  """Load an image from file into a numpy array.

  Puts image into numpy array to feed into tensorflow graph.
  Note that by convention we put it into a numpy array with shape
  (height, width, channels), where channels=3 for RGB.

  Args:
    path: the file path to the image

  Returns:
    uint8 numpy array with shape (img_height, img_width, 3)
  """
  return np.array(Image.open(path))

In [10]:
IMAGES_FOLDER = os.path.join(ROOT, 'data')
TARGET_ANNOTATIONS = os.path.join(ROOT, 'data', 'Annotations')
images_list = os.listdir(IMAGES_FOLDER)
print('Images folder {}'.format(IMAGES_FOLDER))
print('Annotations folder {}'.format(TARGET_ANNOTATIONS))

Images folder gdrive/MyDrive/MIA2/landmark_detections/data
Annotations folder gdrive/MyDrive/MIA2/landmark_detections/data/Annotations


In [19]:
for idx, image_name in enumerate(images_list):
  print('{}/{}. Running inference for {}'.format(idx + 1, len(images_list), image_name))
  img_id = image_name.split('.')[0]
  image_path = os.path.join(IMAGES_FOLDER, image_name)
  image_np = load_image_into_numpy_array(image_path)
  height, width, channels = image_np.shape
  input_tensor = tf.convert_to_tensor(image_np)
  input_tensor = input_tensor[tf.newaxis, ...]

  # run detections
  detections = detect_fn(input_tensor)

  num_detections = int(detections.pop('num_detections'))

  detections = {key: value[0, :num_detections].numpy() for key, value in detections.items()}
  detections['num_detections'] = num_detections
  
  # detection_classes should be ints
  detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

  # create annotation
  # header
  annotation = ET.Element('annotation')
  ET.SubElement(annotation, 'folder').text = 'Images'
  ET.SubElement(annotation, 'filename').text = image_name
  ET.SubElement(annotation, 'path').text = image_path
  source = ET.SubElement(annotation, 'source')
  ET.SubElement(source, 'database').text = 'Unknown'
  size = ET.SubElement(annotation, 'size')
  ET.SubElement(size, 'width').text = str(width)
  ET.SubElement(size, 'height').text = str(height)
  ET.SubElement(size, 'depth').text = str(channels)
  ET.SubElement(annotation, 'segmented').text = '0'

  min_score_thresh = 0.5
  for i in range(len(detections['detection_scores'])):
    if detections['detection_scores'][i] > min_score_thresh:
      object_name = category_index[detections['detection_classes'][i]]['name']
      ymin, xmin, ymax, xmax = detections['detection_boxes'][i]
      xmin = int(xmin * width)
      xmax = int(xmax * width)
      ymin = int(ymin * height)
      ymax = int(ymax * height)

      object_annotation = ET.SubElement(annotation, 'object')
      ET.SubElement(object_annotation, 'name').text = object_name
      ET.SubElement(object_annotation, 'pose').text = 'Unspecified'
      ET.SubElement(object_annotation, 'truncated').text = '0'
      ET.SubElement(object_annotation, 'difficult').text = '0'
      bndbox = ET.SubElement(object_annotation, 'bndbox')
      ET.SubElement(bndbox, 'xmin').text = str(xmin)
      ET.SubElement(bndbox, 'ymin').text = str(ymin)
      ET.SubElement(bndbox, 'xmax').text = str(xmax)
      ET.SubElement(bndbox, 'ymax').text = str(ymax)
  
  tree = ET.ElementTree(annotation)
  tree_root = tree.getroot()
  xmlstr = ET.tostring(tree_root, encoding='utf8', method='xml')

  tree.write(os.path.join(TARGET_ANNOTATIONS, img_id + '.xml'))
  break

1/17. Running inference for img_1914.jpg
0.72027147 0.08162859 0.7808009 0.10216904
